In [ ]:
#import Dependencies
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [ ]:
# Create a path to the csv file and read it into a Pandas DataFrame
movie_api_file_path1 = '../EDA-Project/Data/api_data/clean_movies_api_df1.csv'
api_movie_df1 = pd.read_csv(movie_api_file_path1)
movie_api_file_path2 = '../EDA-Project/Data/api_data/clean_movies_api_df2.csv'
api_movie_df2 = pd.read_csv(movie_api_file_path2)
tmbd_file_path = '../Resources/TMBD Movie Dataset.csv'
tmbd_df = pd.read_csv(tmbd_file_path)

In [ ]:
#Combine both api DataFrames
all__api_movie_data_df = pd.concat([api_movie_df1, api_movie_df2], ignore_index=True)

In [ ]:
#preview api movie data columns
all__api_movie_data_df.columns

In [ ]:
#Drop 'Unnamed: 0' column
all__api_movie_data_df = all__api_movie_data_df.drop('Unnamed: 0', axis=1)

In [ ]:
#Create copy of api_movie_df to clean
clean_api_movie_df = all__api_movie_data_df.copy()

In [ ]:
#Rename Production column
clean_api_movie_df = clean_api_movie_df.rename(columns={"production_companies": 'Production'})

In [ ]:
#Drop columns containing missing values and reset the index
clean_api_movie_df = clean_api_movie_df.dropna()
clean_api_movie_df = clean_api_movie_df.reset_index(drop=True)

In [ ]:
#Preview column data types to see which need to be changed
clean_api_movie_df.dtypes

In [ ]:
#Convert Released column to datetime type
clean_api_movie_df['Released'] =  clean_api_movie_df['Released'].astype('datetime64[ns]')

In [ ]:
#For loop to iterate every row to create lists of items in each column
for i in range(len(clean_api_movie_df)):
    clean_api_movie_df['keywords'][i] = clean_api_movie_df['keywords'][i].split('|')
    clean_api_movie_df['Actors'][i] = clean_api_movie_df['Actors'][i].split(',')
    clean_api_movie_df['Genre'][i] = clean_api_movie_df['Genre'][i].split(',')
    clean_api_movie_df['Production'][i] = clean_api_movie_df['Production'][i].split('|')
    clean_api_movie_df['Director'][i] = clean_api_movie_df['Director'][i].split(',')
    clean_api_movie_df['Writer'][i] = clean_api_movie_df['Writer'][i].split(',')
    clean_api_movie_df['Language'][i] = clean_api_movie_df['Language'][i].split(',')
    clean_api_movie_df['Country'][i] = clean_api_movie_df['Country'][i].split(',')

In [ ]:
#Convert Runtime, ImdbVotes, Box Office, and budget columns to type int
clean_api_movie_df['Runtime'] = clean_api_movie_df['Runtime'].str.replace(' min', '').astype(int)
clean_api_movie_df['imdbVotes'] = clean_api_movie_df['imdbVotes'].str.replace(',', '').astype(int)
clean_api_movie_df['BoxOffice'] = clean_api_movie_df['BoxOffice'].str.replace(',', '')
clean_api_movie_df['BoxOffice'] = clean_api_movie_df['BoxOffice'].str.replace('$', '', regex=True)
clean_api_movie_df['BoxOffice'] = clean_api_movie_df['BoxOffice'].astype(int)
clean_api_movie_df['budget'] = clean_api_movie_df['budget'].astype(int)

In [ ]:
#Create new column to store Rotten Tomatoes score
clean_api_movie_df['Rotten Tomatoes'] = ''

#For loop to interate though each row and pull Rotten Tomatoes score from Ratings column and assign to Rotten Tomatoes column
for i in range(len(clean_api_movie_df)):
    
    if 'Rotten Tomatoes' in clean_api_movie_df['Ratings'][i]:
        clean_api_movie_df.loc[i, 'Rotten Tomatoes'] = clean_api_movie_df['Ratings'][i][99:101]
    else:
        clean_api_movie_df.loc[i, 'Rotten Tomatoes'] = None

In [ ]:
#Drop columns containing missing values and reset the index
clean_api_movie_df = clean_api_movie_df.dropna()
clean_api_movie_df = clean_api_movie_df.reset_index(drop=True)

In [ ]:
#Convert Rotten Tomatoes column to type int
clean_api_movie_df['Rotten Tomatoes'] = clean_api_movie_df['Rotten Tomatoes'].str.replace('%', '').astype(int)

In [ ]:
#Create emtpy lists to store oscars won, oscar nominations, award wins, and award nominations for each movie
oscars_won = []
oscar_nominations = []
award_wins = []
award_nominations = []

#For loop to pull oscars won, oscar nominations, award wins, and award nominations values from awards 
# column and append to corresponding list
for i in range(len(clean_api_movie_df)):
    awards = clean_api_movie_df['Awards'][i].lower()
    awards_string_list = awards.split(' ')
    
    if 'won' in awards_string_list:
        won_index = awards_string_list.index('won')
        oscars_won.append(awards_string_list[won_index + 1])
    else:
        oscars_won.append(0)

    if 'nominated' in awards_string_list:
        nominated_index = awards_string_list.index('nominated')
        oscar_nominations.append(awards_string_list[nominated_index + 2])
    else:
        oscar_nominations.append(0)
        
    if 'wins' in awards_string_list:
        win_index = awards_string_list.index('wins')
        award_wins.append(awards_string_list[win_index - 1])
    elif '1 win &' in awards:
        award_wins.append(1)
    else:
        award_wins.append(0)
    
    if 'nominations' in awards_string_list:
        nominations_index = awards_string_list.index('nominations')
        award_nominations.append(awards_string_list[nominations_index - 1])
    elif '1 nomination' in awards:
        award_nominations.append(1)
    else:
        award_nominations.append(0)

#Create oscar nominations, award wins, and award nominations columns in DataFrame and assign corresponding
clean_api_movie_df['oscar_wins'] = oscars_won
clean_api_movie_df['oscar_nominations'] = oscar_nominations
clean_api_movie_df['award_wins'] = award_wins
clean_api_movie_df['award_nominations'] = award_nominations

In [ ]:
#Convert all values in award wins and award nominations to type str
clean_api_movie_df['award_wins'] = clean_api_movie_df['award_wins'].astype(str)
clean_api_movie_df['award_nominations'] = clean_api_movie_df['award_nominations'].astype(str)

In [ ]:
#For loop to remove any award wins and award nominations values containing 'award' and replace with just number value
for i in range(len(clean_api_movie_df)):
    award_wins = clean_api_movie_df['award_wins'][i]
    award_nominations = clean_api_movie_df['award_nominations'][i]

    if 'award' in award_wins:
        award_wins_string_split = award_wins.split('d')
        award_wins_number = award_wins_string_split[1]
        clean_api_movie_df['award_wins'][i] = award_wins_number
    else:
        pass

    if 'award' in award_nominations:
        award_nominations_string_split = award_nominations.split('d')
        award_nominations_number = award_nominations_string_split[1]
        clean_api_movie_df['award_nominations'][i] = award_nominations_number
    else:
        pass

In [ ]:
#Convert oscars won, oscar nominations, award wins, and award nominations values to type int
clean_api_movie_df['award_wins'] = clean_api_movie_df['award_wins'].astype(int)
clean_api_movie_df['oscar_wins'] = clean_api_movie_df['oscar_wins'].astype(int)
clean_api_movie_df['oscar_nominations'] = clean_api_movie_df['oscar_nominations'].astype(int)
clean_api_movie_df['award_nominations'] = clean_api_movie_df['award_nominations'].astype(int)

In [ ]:
#Create new DataFrame with column rearranged from clean_api_movie_df
all_movie_data_clean_df = clean_api_movie_df.reindex(columns=['Title', 'Released', 'Year', 'Rated', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Language', 'Country', 'Ratings', 'Rotten Tomatoes',
       'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'BoxOffice', 'budget',
       'keywords', 'Production',  'Awards',  'oscar_wins',
       'oscar_nominations', 'award_wins', 'award_nominations'])

In [ ]:
#Preview cleaned movie data
all_movie_data_clean_df.tail()

In [ ]:
#Preview cleaned data column types
all_movie_data_clean_df.dtypes

In [ ]:
#export all_movie_data_clean_df as csv files
all_movie_data_clean_df.to_csv('all_movie_data_clean_df.csv')